In [1]:
import requests
from xml.etree import ElementTree
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import json
import urllib.request
import escher
from escher import Builder
import cobra
import time
from time import sleep

In [2]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [3]:
def createTree(link):
    time.sleep(1)
    response = requests.get(link)
    tree = ElementTree.fromstring(response.content)
    return tree

In [4]:
def createDictFromName(complexList, errors):
    result = {}

    for name in complexList:
        try:
            linkComplex = "https://websvc.biocyc.org/getxml?id=ECOLI:" + name
            treeComplex = createTree(linkComplex)
            treeDictComplex = etree_to_dict(treeComplex)
            result[name] = treeDictComplex
        except Exception as e:
            errors.append((name, e))

    return result

In [5]:
def createDictFromLink(complexList, errors):
    result = {}

    for name in complexList:
        try:
            linkComplex = "https://websvc.biocyc.org/" + name
            treeComplex = createTree(linkComplex)
            treeDictComplex = etree_to_dict(treeComplex)
            result[name] = treeDictComplex
        except Exception as e:
            errors.append((name, e))

    return result

In [6]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

metabolData = np.load(r"../../out/geneRxnVerifData/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

In [7]:
reactions = fluxesWithCaption.keys()

In [8]:
ecReactions = pd.read_csv('reactions.tsv', sep='\t', header=0)['Reaction'].tolist()

In [9]:
foundNames = []
notFound = []
for rxn in ecReactions:
    found = False
    for key in fluxesWithCaption:
        if rxn in key:
            found = True
            foundNames.append((rxn, key))
    if not found:
        notFound.append(rxn)

In [10]:
prev = ""
fluxes = {}
for names in foundNames:
    if prev == names[0]:
        for i in range(len(fluxes[prev])):
            if "(reverse)" in names[1]:
                fluxes[prev][i] -= fluxesWithCaption[names[1]][i]
            else:
                fluxes[prev][i] += fluxesWithCaption[names[1]][i]
    else:
        prev = names[0]
        fluxes[prev] = fluxesWithCaption[names[1]]

In [11]:
with open('../../iJO1366.Central metabolism.json') as json_file:
    data = json.load(json_file)
    
escherRxns = data[1]['reactions']
escherNames = []

for number in escherRxns:
    reactionName = escherRxns[number]['name']
    escherNames.append(reactionName.lower())

In [17]:
errors = []

In [13]:
reactionDict = np.load("acetateReactionDict.npy", allow_pickle=True).tolist()

In [16]:
len(reactionDict.keys())

1720

reactionDict = np.load("glucoseReactionDict.npy", allow_pickle=True).tolist()

In [31]:
ECnames = {}

for key in reactionDict.keys():
    dicte = reactionDict[key]
    names = []
    
    try:
        temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
        if type(temp) is list:
            for item in temp:
                if item['common-name']['#text'] not in names:
                    names.append(item['common-name']['#text'])
        else:
            names.append(dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']['common-name']['#text'])
            
    except Exception as e1:
        try:
            nest = dicte['ptools-xml']['Reaction']['in-pathway']['Reaction']['@resource']
            temp = []
            temp.append(nest)
            new = createDictFromLink(temp, errors)
            temp1 = new[temp[0]]['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
            
            if type(temp1) is list:
                for item in temp1:
                    if item['common-name']['#text'] not in names:
                        names.append(item['common-name']['#text'])
            else:
                names.append(temp['common-name']['#text'])
                
        except Exception as e2:
            errors.append((key, e1, e2))
            
    try:
        temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
        if type(temp) is list:
            for item in temp:
                names += [x['#text'] for x in item['synonym']]
        else:
            names += [x['#text'] for x in temp['synonym']]
            
    except Exception as e1:
        try:
            temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
            if type(temp) is list:
                for item in temp:
                    names.append(item['synonym']['#text'])
            else:
                names.append(item['synonym']['#text'])
        except Exception as e2:
            errors.append((key, e1, e2))
        
    ECnames[key] = names
    

In [32]:
foundNames = {}

for key in ECnames.keys():
    names1 = ECnames[key]
    for name in names1:
        if name.lower() in escherNames:
            foundNames[key] = name[0].upper() + name[1:]
            break

In [62]:
foundNames

{'1PFRUCTPHOSN-RXN': 'Phosphofructokinase',
 'GLYCEROL-1-PHOSPHATASE-RXN': 'Glycerol-3-phosphatase',
 '3PGAREARR-RXN': 'Phosphoglycerate mutase',
 '1TRANSKETO-RXN': 'Transketolase',
 'TRANSALDOL-RXN': 'Transaldolase',
 'GLUC1PADENYLTRANS-RXN': 'Glucose-1-phosphate adenylyltransferase',
 '2PGADEHYDRAT-RXN': 'Enolase',
 'MALATE-DEHYDROGENASE-ACCEPTOR-RXN': 'Malate dehydrogenase',
 'PGLUCISOM-RXN': 'Glucose-6-phosphate isomerase',
 'RXN-15513': 'Phosphoglycerate mutase',
 'CITSYN-RXN': 'Citrate synthase',
 'RIBOKIN-RXN': 'Ribokinase',
 'DLACTDEHYDROGNAD-RXN': 'D-lactate dehydrogenase',
 'METHGLYSYN-RXN': 'Methylglyoxal synthase',
 'CATAL-RXN': 'Catalase',
 'R601-RXN': 'Fumarate reductase',
 '2TRANSKETO-RXN': 'Transketolase',
 'GAPOXNPHOSPHN-RXN': 'Glyceraldehyde-3-phosphate dehydrogenase',
 'PYRUVDEH-RXN': 'Pyruvate dehydrogenase',
 'GLYCOLATEDEHYDRO-RXN': 'Glycolate oxidase',
 'XYLULOKIN-RXN': 'Xylulokinase',
 '6PGLUCONOLACT-RXN': '6-phosphogluconolactonase',
 'GLUCOSE-1-PHOSPHAT-RXN': '

In [63]:
newFluxes = {}

for key in fluxes.keys():
    if key in foundNames.keys():
        newFluxes[foundNames[key]] = fluxes[key][-1]

In [64]:
newFluxes

{'Phosphofructokinase': 0,
 'Glycerol-3-phosphatase': 0,
 'Phosphoglycerate mutase': 0,
 'Transketolase': 400192,
 'Transaldolase': 415326,
 'Glucose-1-phosphate adenylyltransferase': 8598,
 'Enolase': 177197,
 'Malate dehydrogenase': -207626,
 'Glucose-6-phosphate isomerase': -916508,
 'Citrate synthase': 2593,
 'Ribokinase': 0,
 'D-lactate dehydrogenase': 0,
 'Methylglyoxal synthase': 75,
 'Catalase': 0,
 'Fumarate reductase': 209353,
 'Glyceraldehyde-3-phosphate dehydrogenase': 177824,
 'Pyruvate dehydrogenase': 0,
 'Glycolate oxidase': 0,
 'Xylulokinase': 0,
 '6-phosphogluconolactonase': 905030,
 'Glucose-1-phosphatase': 753,
 'Isocitrate lyase': -398381,
 'Acetaldehyde dehydrogenase (acetylating)': -815,
 'Citrate lyase': 160554,
 'Phosphoenolpyruvate carboxylase': 591083,
 'Phosphotransacetylase': -18159,
 'Pyruvate oxidase': 0,
 'Triose-phosphate isomerase': 107773,
 'Ribose-5-phosphate isomerase': -465042,
 'Glycerol dehydrogenase': -1675,
 'Glucose-6-phosphate phosphatase': 0,

In [65]:
additional3 = {'D-glucose transport via PEP:Pyr PTS': 0,
 'FRUpts2': 0,
 'ACt2r': 0,
 'Glucose-6-phosphate isomerase': -916508,
 'Phosphofructokinase': 0,
 'Fructose-bisphosphatase': 10126,
 'Fructose-bisphosphate aldolase': 160869,
 'Triose-phosphate isomerase': 107773,
 'Glyceraldehyde-3-phosphate dehydrogenase': 177824,
 'Phosphoglycerate kinase': -177822,
 'Phosphoglycerate mutase': 0,
 'Phosphoglycerate mutase': -177087,
 'Enolase': 177197,
 'Pyruvate kinase': 0,
 'Phosphoenolpyruvate synthase': 446062}

additional = {'D-glucose transport via PEP:Pyr PTS': 0,
 'FRUpts2': 0,
 'Glucose-6-phosphate isomerase': -3665600,
 'Phosphofructokinase': 0,
 'Fructose-bisphosphatase': 38308,
 'Fructose-bisphosphate aldolase': 385734,
 'Triose-phosphate isomerase': 107701,
 'Glyceraldehyde-3-phosphate dehydrogenase': 177895,
 'Phosphoglycerate kinase': -711572,
 'Phosphoglycerate mutase': 0,
 'Phosphoglycerate mutase': -708632,
 'Enolase': 177268,
 'Pyruvate kinase': 0,
 'Phosphoenolpyruvate synthase': 445971}

additional2 = {'D-glucose transport via PEP:Pyr PTS': 516124,
 'FRUpts2': 0,
 'Glucose-6-phosphate isomerase': 395206,
 'Phosphofructokinase': 90275,
 'Fructose-bisphosphatase': 9774,
 'Fructose-bisphosphate aldolase': 151536,
 'Triose-phosphate isomerase': -491270,
 'Glyceraldehyde-3-phosphate dehydrogenase': 517023,
 'Phosphoglycerate kinase': -1034042,
 'Phosphoglycerate mutase': 0,
 'Phosphoglycerate mutase': -1032508,
 'Enolase': 516362,
 'Pyruvate kinase': 0,
 'Phosphoenolpyruvate synthase': 0}

for x in additional.keys():
    newFluxes[x] = additional[x]

for x in additional2.keys():
    newFluxes[x] = additional2[x]

In [66]:
for x in additional3.keys():
    newFluxes[x] = additional3[x]

In [67]:
newFluxes

{'Phosphofructokinase': 0,
 'Glycerol-3-phosphatase': 0,
 'Phosphoglycerate mutase': -177087,
 'Transketolase': 400192,
 'Transaldolase': 415326,
 'Glucose-1-phosphate adenylyltransferase': 8598,
 'Enolase': 177197,
 'Malate dehydrogenase': -207626,
 'Glucose-6-phosphate isomerase': -916508,
 'Citrate synthase': 2593,
 'Ribokinase': 0,
 'D-lactate dehydrogenase': 0,
 'Methylglyoxal synthase': 75,
 'Catalase': 0,
 'Fumarate reductase': 209353,
 'Glyceraldehyde-3-phosphate dehydrogenase': 177824,
 'Pyruvate dehydrogenase': 0,
 'Glycolate oxidase': 0,
 'Xylulokinase': 0,
 '6-phosphogluconolactonase': 905030,
 'Glucose-1-phosphatase': 753,
 'Isocitrate lyase': -398381,
 'Acetaldehyde dehydrogenase (acetylating)': -815,
 'Citrate lyase': 160554,
 'Phosphoenolpyruvate carboxylase': 591083,
 'Phosphotransacetylase': -18159,
 'Pyruvate oxidase': 0,
 'Triose-phosphate isomerase': 107773,
 'Ribose-5-phosphate isomerase': -465042,
 'Glycerol dehydrogenase': -1675,
 'Glucose-6-phosphate phosphatas

In [68]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [69]:
escher.list_available_models()

[{'organism': 'Saccharomyces cerevisiae', 'model_name': 'iMM904'},
 {'organism': 'Homo sapiens', 'model_name': 'RECON1'},
 {'organism': 'Escherichia coli', 'model_name': 'e_coli_core'},
 {'organism': 'Escherichia coli', 'model_name': 'iJO1366'}]

In [3]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
    reaction_data={r: v for r, v in zip(newFluxes.keys(), newFluxes.values())},
)

NameError: name 'newFluxes' is not defined

In [ ]:
builder

In [26]:
ECnames['PGLUCISOM-RXN']

['glucose-6-phosphate isomerase',
 'phosphoglucose isomerase',
 'D-glucose-6-phosphate-ketol-isomerase']

In [27]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][0]['common-name']['#text']

'glucose-6-phosphate isomerase'

In [28]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][1]['common-name']['#text']

'glucose-6-phosphate isomerase'

In [29]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][0]['synonym']

[{'@datatype': 'string', '#text': 'phosphoglucose isomerase'},
 {'@datatype': 'string', '#text': 'D-glucose-6-phosphate-ketol-isomerase'}]

In [30]:
def getNames(liste, names):
    for item in liste:
        if item['common-name']['#text'] not in names:
            names.append(item['common-name']['#text'])
    for item in liste:
        names
        

In [31]:
np.save("glucoseReactionDict.npy", reactionDict)